In [ ]:
import numpy as np
#from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense

#connect drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/Strukov0107/TerVer_Ana_MatStat.git

In [ ]:
# Каталог с данными для обучения
train_dir = '/content/TerVer_Ana_MatStat/PH_Generator/Dataset'
#/content/TerVer_Ana_MatStat/PH_Generator/Dataset
# Каталог с данными для проверки
val_dir = '/content/TerVer_Ana_MatStat/PH_Generator/Dataset'
# Каталог с данными для тестирования
test_dir = '/content/TerVer_Ana_MatStat/PH_Generator/Dataset'
#
exc_path = '/content/TerVer_Ana_MatStat/PH_Generator/What_Is'

# Размеры изображения
#img_width, img_height = 1191, 1684
img_width, img_height = 119, 168
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 20
# Размер мини-выборки
batch_size = 25
# Количество изображений для обучения
nb_train_samples = 500
# Количество изображений для проверки
nb_validation_samples = 50
# Количество изображений для тестирования
nb_test_samples = 50




datagen = ImageDataGenerator(rescale=1. /255)
#генератор данных для тестирования на основе изображений из каталога
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

#генератор данных для тестирования на основе изображений из каталога
#datagen = ImageDataGenerator(rescale=1. /255)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

#генератор для проверки на основе изображений из каталога
#datagen = ImageDataGenerator(rescale=1. /255)

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)



In [ ]:
#создаём нейронку
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
#model.add(Dropout(0.05))
model.add(Dense(1))
model.add(Activation('sigmoid'))

#компилируем нейронную сеть
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size
)

# оцениваем качество работы сети с помощью генератора
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

print ("Accurancy by test data : %.2f%%" % (scores[1]*100))

In [ ]:
#генератор данных для исследования нового неизвестного образца
datagen = ImageDataGenerator(rescale=1. /255)

exc_generator = datagen.flow_from_directory(
    exc_path,
    target_size=(img_width, img_height),
    batch_size = 1,
    class_mode = 'binary'
)

for datase_pic, dataset_indx in exc_generator:
  x = datase_pic
  y = dataset_indx
#  print (datase_pic)
  print (dataset_indx)
  break

predictions_single = model.predict(x)
predictions_single

In [ ]:
print(np.argmax(predictions_single))